In [18]:
# Import the dependencies

import json
import pandas as pd
import numpy as np

In [19]:
# Get data directory 

from local_config import data_dir

In [20]:
# Extract the Movielens and ratings data

movies_metadata = pd.read_csv(f'{data_dir}movies_metadata.csv',low_memory=False)
ratings = pd.read_csv(f'{data_dir}ratings.csv')

In [21]:
# Extract the Wikipedia data

with open(f'{data_dir}/wikipedia.movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)

In [22]:
# Convert the wiki_movies json data to a dataframe

wiki_movies_df = pd.DataFrame(wiki_movies_raw)

In [23]:
# Get list of movie json objects which satisfy the following conditions:
# - one of the keys, Director or 'Directed by,' is present
# - the imdb_link key is present
# - the 'No. of episodes' key is not present

wiki_movies = [movie for movie in wiki_movies_raw
                if ('Director' in movie or 'Directed by' in movie)
                and 'imdb_link' in movie
                and 'No. of episodes' not in movie]

In [39]:
len(wiki_movies)

7076

In [24]:
# Load the wiki_movies json objects list into a dataframe

wiki_df = pd.DataFrame(wiki_movies)

In [25]:
# This function does the following:
# Phase 1 - moves alternate title keys to alt_titles dict object. It does so as follows:
# - scans the movie dict parameter for alternate title keys
# - adds the alternate title keys into a new dict object, alt_titles
# - removes the alternate titles items from the movie dict
# 

# Phase II - renames the keys in the movie dict to match those in the kaggle data

# Return:
# The cleaned movie dict

def clean_movie(movie):
    
    # Phase I
    movie = dict(movie)  # create a non-destructive copy
    alt_titles = {}
    
    # scan the movie dict parameter for alternate title keys
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
          
        if key in movie:
            # add the alternate title keys into a new dict object, alt_titles
            alt_titles[key] = movie[key]
            
            # remove the key,value item from the movie dict
            movie.pop(key)
    
    # if any alternate titles dict has any items, add the alt_titles dict to the movie dict using key, 'alt_titles'
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
    
    # Phase II - rename the keys in the movie dict to match those in the kaggle data
    
    # This inner function changes a column name from old_name and new_name
    #
    # Parameters:
    # old_name - old key name
    # new_name - new key name
    
    def change_column_name(old_name, new_name):
        # If the key old_name exists in the movie dict
        if old_name in movie:
            
            # place the value of movie[old_name] into movie[new_name] and delete the old_name item
            movie[new_name] = movie.pop(old_name)
    
    # Perform the following key name changes 
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')
    
    return movie

In [26]:
# Clean the movies in in the wiki_movies list

clean_movies = [clean_movie(movie) for movie in wiki_movies]

In [40]:
len(clean_movies)

7076

In [41]:
# Redefine wiki_movies_df using the clean_movies dict list
wiki_movies_df = pd.DataFrame(clean_movies)

In [42]:
# Extract the imdb_id from using regular expression
wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')

## Remove next cell

In [43]:
imdb_df = pd.DataFrame(wiki_movies_df['imdb_id']).dropna()
# imdb_df[imdb_df.duplicated()]["imdb_id"]

duplicates = wiki_movies_df.loc[wiki_movies_df.imdb_id.isin(imdb_df[imdb_df.duplicated()]["imdb_id"])].sort_values("imdb_id")

dups = []
for item in duplicates["imdb_id"]:
    if not (item in dups):
        dups.append(item)
        
dups

['tt0099180',
 'tt0099816',
 'tt0101414',
 'tt0102432',
 'tt0103002',
 'tt0103923',
 'tt0105226',
 'tt0105616',
 'tt0107978',
 'tt0107983',
 'tt0109266',
 'tt0113646',
 'tt0113855',
 'tt0115819',
 'tt0118577',
 'tt0119256',
 'tt0120241',
 'tt0120338',
 'tt0120915',
 'tt0165831',
 'tt0198781',
 'tt0226168',
 'tt0259446',
 'tt0265307',
 'tt0266543',
 'tt0273982',
 'tt0290212',
 'tt0296192',
 'tt0321780',
 'tt0498381',
 'tt0795351',
 'tt0844286',
 'tt1107319',
 'tt1440379',
 'tt1781840',
 'tt1885300',
 'tt1930294',
 'tt1935896',
 'tt2023765',
 'tt2083379',
 'tt2103264',
 'tt2140577']

In [50]:
wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)

In [52]:
wiki_movies_df.shape

(7033, 41)

In [61]:
# Keep only those columns which have less than 90% null values

wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]